In [1]:
import os
import logging
import json

import main

/home/ubuntu/ThinkLogits/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%cd ..

/home/ubuntu/ThinkLogits


/home/ubuntu/ThinkLogits/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model, tokenizer, model_name, device = main.load_model_and_tokenizer(model_path)

2025-04-14 10:29:30,648 - INFO - CUDA is available. Using GPU.
2025-04-14 10:29:30,651 - INFO - Loading model and tokenizer: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B onto cuda
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
2025-04-14 10:29:34,130 - INFO - Model and tokenizer loaded successfully.


In [ ]:
main.generate_dataset_completions(
    model = model,
    tokenizer = tokenizer,
    model_name = model_name,
    device = device,
    hint_types = ["sycophancy"],
    batch_size = 5,
    max_new_tokens = None,
    n_questions = 150
    )

2025-04-14 10:29:35,567 - INFO - Using chat template: User: {instruction}
Assistant:
2025-04-14 10:29:35,569 - INFO - --- Processing dataset for hint type: sycophancy ---
2025-04-14 10:29:35,598 - INFO - Generating completions for sycophancy...
2025-04-14 10:29:35,599 - INFO - Using max_new_tokens: 2048
2025-04-14 10:29:35,600 - INFO - Processing batch 1/30 (Size: 5, QIDs: 0-4)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-14 10:29:50,308 - INFO - Processing batch 2/30 (Size: 5, QIDs: 5-9)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


KeyboardInterrupt: 